In [ ]:
from perses.protocols.nonequilibrium_cycling import SimulationUnit, ResultUnit
from gufe import ChemicalSystem, SmallMoleculeComponent, ProteinComponent, SolventComponent


In [ ]:
protein_comp = ProteinComponent.from_pdb_file("Tyk2_protein.pdb")
ligand_a_comp = SmallMoleculeComponent.from_sdf_file('ejm_31.sdf')
ligand_b_comp = SmallMoleculeComponent.from_sdf_file('jmc_30.sdf')
state_a_full = {'protein': protein_comp, 'ligand': ligand_a_comp}
state_b_full = {'protein': protein_comp, 'ligand': ligand_b_comp}
state_a_ligand = {'ligand': ligand_a_comp}
state_b_ligand = {'ligand': ligand_b_comp}

system_a = ChemicalSystem(components=state_a_ligand)
system_b = ChemicalSystem(components=state_b_ligand)


In [ ]:
# mapping class implementation

import gufe
from typing import Any, Optional
import numpy as np
from numpy.typing import NDArray
import json

class LigandAtomMapping(gufe.AtomMapping):
    """Simple container with the mapping between two Molecules

    """
    def __init__(
        self,
        componentA: SmallMoleculeComponent,
        componentB: SmallMoleculeComponent,
        componentA_to_componentB: dict[int, int],
        annotations: Optional[dict[str, Any]] = None,
    ):
        """
        Parameters
        ----------
        componentA, componentB : SmallMoleculeComponent
          the ligand molecules on either end of the mapping
        componentA_to_componentB : dict[int, int]
          correspondence of indices of atoms between the two ligands
        annotations : dict[str, Any]
          Mapping of annotation identifier to annotation data. Annotations may
          contain arbitrary JSON-serializable data. Annotation identifiers
          starting with ``ofe-`` may have special meaning in other parts of
          OpenFE. ``score`` is a reserved annotation identifier.
        """
        super().__init__(componentA, componentB)
        self._compA_to_compB = componentA_to_componentB

        if annotations is None:
            # TODO: this should be a frozen dict
            annotations = {}

        self._annotations = annotations

    @classmethod
    def _defaults(self):
        return {}

    @property
    def componentA_to_componentB(self) -> dict[int, int]:
        return dict(self._compA_to_compB)

    @property
    def componentB_to_componentA(self) -> dict[int, int]:
        return {v: k for k, v in self._compA_to_compB.items()}

    @property
    def componentA_unique(self):
        return (i for i in range(self.componentA.to_rdkit().GetNumAtoms())
                if i not in self._compA_to_compB)

    @property
    def componentB_unique(self):
        return (i for i in range(self.componentB.to_rdkit().GetNumAtoms())
                if i not in self._compA_to_compB.values())

    @property
    def annotations(self):
        # return a copy (including copy of nested)
        return json.loads(json.dumps(self._annotations))

    def _to_dict(self):
        """Serialize to dict"""
        return {
            # openff serialization doesn't go deep, so stringify at this level
            'componentA': json.dumps(self.componentA.to_dict(), sort_keys=True),
            'componentB': json.dumps(self.componentB.to_dict(), sort_keys=True),
            'componentA_to_componentB': self._compA_to_compB,
            'annotations': json.dumps(self.annotations),
        }

    @classmethod
    def _from_dict(cls, d: dict):
        """Deserialize from dict"""
        # the mapping dict gets mangled sometimes
        mapping = d['componentA_to_componentB']
        fixed = {int(k): int(v) for k, v in mapping.items()}

        return cls(
            componentA=SmallMoleculeComponent.from_dict(
                json.loads(d['componentA'])),
            componentB=SmallMoleculeComponent.from_dict(
                json.loads(d['componentB'])),
            componentA_to_componentB=fixed,
            annotations=json.loads(d['annotations'])
        )

    def __repr__(self):
        return (f"{self.__class__.__name__}(componentA={self.componentA!r}, "
                f"componentB={self.componentB!r}, "
                f"componentA_to_componentB={self._compA_to_compB!r}, "
                f"annotations={self.annotations!r})")

    def _ipython_display_(self, d2d=None):  # pragma: no-cover
        """
        Visualize atom mapping in a Jupyter Notebook.

        Parameters
        ---------
        d2d : :class:`rdkit.Chem.Draw.rdMolDraw2D.MolDraw2D`
            If desired specify an instance of a MolDraw2D object.
            Default ``None`` will use the MolDraw2DCairo backend.

        Returns
        -------
        Image: IPython.core.display.Image
            Image of the atom map
        """
        from IPython.display import Image, display

        return display(Image(draw_mapping(self._compA_to_compB,
                                          self.componentA.to_rdkit(),
                                          self.componentB.to_rdkit(), d2d)))

    def draw_to_file(self, fname: str, d2d=None):
        """
        Save atom map visualization to disk

        Parameters
        ---------
        d2d : :class:`rdkit.Chem.Draw.rdMolDraw2D.MolDraw2D`
            If desired specify an instance of a MolDraw2D object.
            Default ``None`` will write a .png file using the MolDraw2DCairo
            backend.

        fname : str
            Name of file to save atom map
        """
        data = draw_mapping(self._compA_to_compB, self.componentA.to_rdkit(),
                            self.componentB.to_rdkit(), d2d)
        if type(data) == bytes:
            mode = "wb"
        else:
            mode = "w"

        with open(fname, mode) as f:
            f.write(draw_mapping(self._compA_to_compB, self.componentA.to_rdkit(),
                                 self.componentB.to_rdkit(), d2d))

    def with_annotations(self, annotations: dict[str, Any]):
        """Create an new mapping based on this one with extra annotations.

        Parameters
        ----------
        annotations : dict[str, Any]
            Annotation update for this mapping. New annotation keys will be
            added to the annotations dict; existing keys will be replaced by
            the data provided here.
        """
        return self.__class__(
            componentA=self.componentA,
            componentB=self.componentB,
            componentA_to_componentB=self._compA_to_compB,
            annotations=dict(**self.annotations, **annotations)
        )

    def get_distances(self) -> NDArray[np.float64]:
        """Return the distances between pairs of atoms in the mapping"""
        dists = []
        molA = self.componentA.to_rdkit().GetConformer()
        molB = self.componentB.to_rdkit().GetConformer()
        for i, j in self._compA_to_compB.items():
            dA = molA.GetAtomPosition(i)
            dB = molB.GetAtomPosition(j)
            dists.append(dA.Distance(dB))

        return np.array(dists)

In [ ]:
# Build gufe mapping object
from gufe.mapping.atom_mapping import AtomMapping
# Extracted from perses AtomMapper
mapping_dict = {0: 0,
 1: 1,
 2: 2,
 3: 3,
 4: 4,
 5: 5,
 6: 6,
 7: 7,
 8: 8,
 9: 9,
 10: 10,
 11: 11,
 12: 12,
 13: 13,
 14: 14,
 15: 15,
 16: 16,
 17: 17,
 18: 18,
 23: 20,
 26: 22,
 27: 23,
 28: 24,
 29: 25,
 30: 26,
 31: 27,
 32: 28,
 33: 29}
mapping = LigandAtomMapping(
    componentA=system_a.components['ligand'],
    componentB=system_b.components['ligand'],
    componentA_to_componentB=mapping_dict)

In [ ]:
from openff.units import unit
# Build Settings gufe object
from gufe.settings.models import (
    Settings, 
    ThermoSettings, 
    ForcefieldSettings,
)
from perses.protocols.settings import NonEqCyclingSettings
settings = Settings(
    settings_version=0,
    forcefield_file="foobar.xml", 
    forcefield_settings=ForcefieldSettings(),
    thermo_settings=ThermoSettings(temperature=300*unit.kelvin),
    protocol_settings=NonEqCyclingSettings(),
)
# non_eq_settings = settings.NonEqCyclingSettings()

In [ ]:
settings.json()

In [ ]:
settings.dict()

In [ ]:
# Debugging SimulationUnit
simulation = SimulationUnit(state_a=system_a, state_b=system_b, mapping=mapping, settings=settings)
result = simulation.execute(shared='/tmp/', **simulation.inputs)


In [ ]:
result.ok()

In [ ]:
print(result.traceback)

In [ ]:
# GatherUnit -- Results
neq_results = ResultUnit(phase="vacuum")
# result = neq_results.execute(shared='/tmp/', **neq_results.inputs)
result = neq_results.execute(shared='/tmp/', **neq_results.inputs)

In [ ]:
result.outputs

In [ ]:
# Running the NonEq Cycling Protocol
from perses.protocols.nonequilibrium_cycling import NonEquilibriumCyclingProtocol
from gufe.protocols.protocoldag import execute_DAG
neq_cycling = NonEquilibriumCyclingProtocol(settings)
dag_result = execute_DAG(neq_cycling.create(stateA=system_a, 
                                            stateB=system_b, 
                                            mapping=mapping,
                                           )
                        )

In [ ]:
dag_result.ok()

In [ ]:
dag_result.protocol_unit_failures

In [ ]:
failure = dag_result.protocol_unit_failures[0]

In [ ]:
print(failure.traceback)

In [ ]:
dag_result.protocol_unit_results[0].outputs

In [ ]:
dag_result.protocol_unit_results[1].outputs

In [ ]:
dag_result.terminal_protocol_unit_results()[0].outputs

In [ ]:
protocol_result = neq_cycling.gather([dag_result])

In [ ]:
protocol_result.data

In [ ]:
result_graph = dag_result.result_graph

In [ ]:
result_graph.nodes

In [ ]:
dag_result.graph.nodes